# Assignment 3 - Security and Privacy of Machine Learning
### By Daan Brugmans (s1080742)

This notebook contains the implementation of Assignment 2 for the Radboud University course [Security and Privacy of Machine Learning](https://www.ru.nl/courseguides/science/vm/osirislinks/imc/nwi-imc069/).
The topic at hand is to execute, analyze, and defend against poisoning attacks on convolutional neural networks within a *Federated Learning (FL)* setting.
Specifically, this notebook wil look at [BadNet Attacks](https://arxiv.org/abs/1708.06733) and [Blend Attacks](https://arxiv.org/pdf/1712.05526). 
A model will be trained using FL, and some of the participants will poison their local model during the FL process.
This notebook will look at a variety of consequences of poisoned models in an FL setup.

This notebook is divided into two parts.
In the first part, we will set up and define all components that we need in order to answer the questions posed in the assignment.
In the second part, we will answer the questions in a Q&A-style by providing the question, the code that will give us an answer, and the answer itself.

Additionally, there is a subdirectory called `src`.
This folder contains a set of Python files directly taken from the week 11 tutorial (Federated Learning, dr. Picek).
Many aspects of the FL setting are already implemented in these files and I will make use of them.

This notebook should show all results without being needed to run.
However, if you do want to run this notebook, I have provided a `requirements.txt` that you can use to install all the required packages.
You can also find this notebook with [this URL](https://github.com/daanbrugmans/ru-security-and-privacy-of-machine-learning-23-24/blob/main/assignments/assignment-3/assignment-3.ipynb).

## Part 1: Setup

### 1.1 Imports
We will use PyTorch as the main environment for our deep learning endeavors. To this end, we will use the `torch` and `torchvision` packages.

In [1]:
import src.Utils
import src.PoisoningUtils
import src.TrainingUtils
import src.ModelUtils
from src.ResNet18Light import ResNet18Light
from src.DataLoader import MyDataLoader
from src.ModelStateDictNames import NAMES_OF_AGGREGATED_PARAMETERS

import random
import gc
from abc import ABC, abstractmethod
from copy import copy

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn.utils.prune 
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision
from torchvision import transforms

### 1.2 Preparation & Settings
Some preparatory code is performed here: we set a seed for `torch`, `numpy`, and `random` for reproducibility, and we set the device on which we will perform our model training. We also define a function for visualizing a batch of images, so that we can visually check if a backdoor has been executed successfully.

We also define some settings related to Federated Learning.
These settings can be tweaked during the assignment and make sure that the already implemented FL code in `src` works correctly.
The settings are thusly taken from the week 11 tutorial (Federated Learning, dr. Picek).

In [15]:
class FLSettings:
    total_client_count = 6
    malignant_client_count = 2
    benign_client_count = total_client_count - malignant_client_count
    
    iid_rate = 0.9
    samples_per_client = 384
    batch_size = 64
    test_batch_size = 1024
        
    global_aggregation_rounds = 5
    local_epochs = 2
    client_count_per_round = 5
    
    malignant_client_poisoning_rate = 0.5
    backdoor_target_class = 0
    
    data_mean = torch.from_numpy(np.array([0.4914, 0.4822, 0.4465]))
    data_std = torch.from_numpy(np.array([0.2023, 0.1994, 0.2010]))
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(data_mean, data_std),
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(data_mean, data_std),
    ])

def visualize_batch(images: torch.Tensor, labels: torch.Tensor):
    """Visualizes a batch of images.
    
    Taken from week 9 notebook (Backdoor Defenses, dr. Picek) and refactored."""
    
    # Making sure we can view the images
    images = images.detach().numpy()
    images = images*255
    images = [image.astype(np.uint8) for image in images]
    images = [np.moveaxis(image, source=0, destination=-1) for image in images]
    
    # Plot the images in the batch, along with the corresponding labels
    fig = plt.figure(figsize=(25, 4))
    for idx in np.arange(20):
        ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
        ax.imshow(images[idx], cmap='viridis')
        # Print out the correct label for each image
        # .item() gets the value contained in a Tensor
        ax.set_title(str(labels[idx].item()))
    
    plt.show()

def set_global_seed(seed):
    """Sets the same seed for varying libraries.
    
    Taken from week 5 lab notebook (Evasion Attacks (Defenses), dr. Picek)"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        
set_global_seed(3131)
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if str(device) == "cuda":
    print(torch.cuda.get_device_name())

cuda
NVIDIA GeForce GTX 1050


### 1.3 Attacks
Here, we define the attacks that we will perform on our neural network.
We define two backdoor attack models: the BadNet Attack and the Blend Attack.
I made the implementation for the Blend Attack myself, using template code from the week 8 tutorial.
The BadNet Attack uses the backdoor implemented in `src.PoisoningUtils.py`.

In [11]:
class Attack(ABC):
    def __init__(self) -> None:
        self.attack_name: str
        self.source_label: int
        self.target_label: int
    
    @abstractmethod
    def execute(self, image: torch.Tensor):
        raise NotImplementedError("The Attack base class is abstract. Please use an implementation of an Attack.")
    
class BadNetAttack(Attack):
    def __init__(self, source_label: int, target_label: int) -> None:
        super().__init__()
        
        self.attack_name = "BadNet Attack"
        
        self.source_label = source_label
        self.target_label = target_label
        
    def execute(self, image: torch.Tensor) -> torch.Tensor:
        backdoored_image, _ = src.PoisoningUtils.poison_single_image(
            image=image,
            label=self.target_label,
            BACKDOOR_TARGET_CLASS=self.target_label,
            MEAN=FLSettings.data_mean,
            STD_DEV=FLSettings.data_std
        )
        
        return backdoored_image

class BlendAttack(Attack):
    def __init__(self, source_label: int, target_label: int) -> None:
        super().__init__()
        
        self.attack_name = "Blend Attack"
        
        self.source_label = source_label
        self.target_label = target_label
        
        self.blend_image = Image.open("./hello_kitty.jpg")
        self.blend_image = self.blend_image.resize((32, 32))
        self.blend_image = np.array(self.blend_image) / 255
        self.blend_image = np.moveaxis(self.blend_image, source=-1, destination=0)
        
    def execute(self, image: torch.Tensor) -> torch.Tensor:
        backdoored_image = image + self.blend_image
        backdoored_image = backdoored_image / 2
        backdoored_image = backdoored_image.to(torch.float32)
        
        return backdoored_image

### 1.4 Data
We define a backdoored version of the CIFAR-10 dataset. This backdoored version of CIFAR-10 takes an `Attack` object and uses it to backdoor the CIFAR-10 data.

We load the CIFAR-10 dataset using the function `get_cifar10_dataloaders`. When called, the function returns 3 `DataLoader` objects: for the train set, the validation set, and the test set respectively. If we pass an `Attack` object to the function, it will return dataloaders of a backdoored CIFAR-10. Otherwise, it returns the regular, clean CIFAR-10.

In [4]:
class BackdooredCIFAR10(Dataset):
        def __init__(self, backdoor: Attack, train: bool) -> None:
            super().__init__()
            
            self.epsilon = 0.08
                        
            if train:
                self.clean_cifar10 = torchvision.datasets.CIFAR10(root="d:/Datasets", download=True, train=train, transform=FLSettings.transform_train)
            else:
                self.clean_cifar10 = torchvision.datasets.CIFAR10(root="d:/Datasets", download=True, train=train, transform=FLSettings.transform_test)
            self.clean_cifar10_loader = DataLoader(self.clean_cifar10, batch_size=1, shuffle=True)
            
            self.backdoor = backdoor
            self.backdoored_cifar10 = []
            self.backdoored_sample_count = round(len(self.clean_cifar10) * self.epsilon, 0)
                        
            if train:
                self._backdoor_train()
            else:
                self._backdoor_test()
                                                                
        def __len__(self) -> int:
            return len(self.backdoored_cifar10)
        
        def __getitem__(self, index):
            return self.backdoored_cifar10[index]
        
        def _backdoor_train(self):
            # If attack is source agnostic
            if self.backdoor.source_label is None:
                for index, (image, label) in enumerate(self.clean_cifar10_loader):
                    label = label.item()
                    image = torch.squeeze(image, 0)
                    
                    # If the image belongs to the subset of images we want to backdoor
                    if index < self.backdoored_sample_count:
                        backdoored_image = self.backdoor.execute(image)
                        self.backdoored_cifar10.append((backdoored_image, self.backdoor.target_label))
                    # If the image does not belong to the subset of images we want to backdoor
                    else:
                        self.backdoored_cifar10.append((image, label))
                    
            # If attack is source specific
            else:
                for index, (image, label) in enumerate(self.clean_cifar10_loader):
                    label = label.item()
                    image = torch.squeeze(image, 0)
                
                    # If the image belongs to the subset of images we want to backdoor
                    if index < self.backdoored_sample_count:
                        backdoored_image = self.backdoor.execute(image)
                    
                        if label == self.backdoor.source_label:
                            self.backdoored_cifar10.append((backdoored_image, self.backdoor.target_label))
                        else:
                            self.backdoored_cifar10.append((backdoored_image, label))  
                    # If the image does not belong to the subset of images we want to backdoor
                    else:
                        self.backdoored_cifar10.append((image, label))
        
        def _backdoor_test(self):
            for image, label in iter(self.clean_cifar10_loader):
                label = label.item()
                image = torch.squeeze(image, 0)
                adversarial_image = self.backdoor.execute(image)
                
                self.backdoored_cifar10.append((adversarial_image, label))

def get_cifar10_dataloaders(backdoor: Attack = None, train_split=0.8) -> tuple[DataLoader, DataLoader, DataLoader]:
    """Returns DataLoader objects for a train, validation, and test set of the CIFAR-10 dataset. If an `Attack` object is passed, it will backdoor the data using the object first."""
    
    if backdoor is None:
        cifar10_dataset_train_val = torchvision.datasets.CIFAR10(root="d:/Datasets", train=True, download=True, transform=FLSettings.transform_train)
        cifar10_dataset_test = torchvision.datasets.CIFAR10(root="d:/Datasets", train=False, download=True, transform=FLSettings.transform_test)
    else:
        cifar10_dataset_train_val = BackdooredCIFAR10(backdoor, train=True)
        cifar10_dataset_test = BackdooredCIFAR10(backdoor, train=False)
    
    train_size = int(len(cifar10_dataset_train_val) * train_split)
    val_size = int(len(cifar10_dataset_train_val) - train_size)
    cifar10_dataset_train, cifar10_dataset_val = random_split(cifar10_dataset_train_val, [train_size, val_size])
    
    cifar10_dataloader_train = DataLoader(cifar10_dataset_train, batch_size=128, shuffle=True)
    cifar10_dataloader_val = DataLoader(cifar10_dataset_val, batch_size=128, shuffle=False)
    cifar10_dataloader_test = DataLoader(cifar10_dataset_test, batch_size=128, shuffle=False)
    
    return cifar10_dataloader_train, cifar10_dataloader_val, cifar10_dataloader_test

### 1.5 Neural Network
The following code block consists of a class definition for the `NeuralModel`.
This class is a collection of all processes and objects that are needed for training a neural network.
It contains a pre-trained instance of the `ResNet18Light` network, as defined in `src/ResNet18Light.py`, as well as the network's loss function, its optimizer, the number of training epochs, and dataloaders for the train, validation, and test sets.
It also contains functions for training and testing the neural network.
Finally, it contains a function that can be used to plot the train/validation loss, accuracy, and ASR for the most recent training run.

I have chosen to implement it this way, so that all code related to the neural network and its architecture is encapsulated within a single class.
In my opinion, this makes performing varying attacks very clean: with only a few rows of code, I am able to instantiate, train, and test a new model.
This makes the experiments easy to read and hides away set implementation details.

In [5]:
class NeuralModel:
    def __init__(self, network_name: str = "", epochs: int = 30, attack: Attack = None) -> None:
        pretrained_weights_file = "./R0099.pt"
        self.neural_network_state_dict = torch.load(pretrained_weights_file, map_location=device)
        self.neural_network = ResNet18Light(network_name).to(device)
        self.neural_network.load_state_dict(self.neural_network_state_dict)
        
        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.AdamW(self.neural_network.parameters(), lr=0.001)
        self.epochs = epochs
        
        if attack is None:
            self.train_data, self.val_data, self.test_data = get_cifar10_dataloaders()
            self.attack = None
        else:
            self.train_data, self.val_data, self.test_data = get_cifar10_dataloaders(backdoor=attack)
            self.attack = attack
        
        self.history = None
        
    def train(self):
        """Train the network."""
        
        # Keep record of loss and accuracy metrics for most recent training procedure
        self.history = {
            "Train Type": "Clean" if self.attack is None else f"Adversarial ({self.attack.attack_name})",
            "Train Loss": [],
            "Validation Loss": [],
            "Train Accuracy": [],
            "Validation Accuracy": []
        }
        
        for epoch in range(self.epochs):
            print(f"Started Epoch {epoch + 1}")
            
            self.neural_network.train()
            
            # Train
            print(" Training...")
            
            train_batch_losses = []
            train_batch_accuracies = []
            
            for images, targets in tqdm(self.train_data):
                images = images.to(device)
                targets = targets.to(device)
                
                predictions = self.neural_network(images)
                
                # Calculate train loss and backpropagate
                train_batch_loss = self.loss_function(predictions, targets)
                train_batch_losses.append(train_batch_loss)
                train_batch_loss.backward()
                
                # Move predictions and labels to cpu for accuracy calculation
                predictions = torch.max(predictions, dim=1)[1]
                predictions = predictions.cpu().detach().numpy()
                targets = targets.cpu().detach().numpy()
                
                # Calculate train accuracy
                train_batch_accuracy = sklearn.metrics.accuracy_score(y_pred=predictions, y_true=targets)
                train_batch_accuracies.append(train_batch_accuracy)
                
                # Step optimizer and clear gradients
                self.optimizer.step()
                self.optimizer.zero_grad()
            
            # Calculate epoch loss and accuracy    
            train_epoch_loss = float(torch.stack(train_batch_losses).mean())
            self.history["Train Loss"].append(train_epoch_loss)
            
            train_epoch_accuracy = np.mean(train_batch_accuracies)
            self.history["Train Accuracy"].append(train_epoch_accuracy)
                
            # Validate
            print(" Validating...")
            
            val_batch_losses = []
            val_batch_accuracies = []
            
            with torch.no_grad():
                self.neural_network.eval()
                
                for images, targets in tqdm(self.val_data):
                    images = images.to(device)
                    targets = targets.to(device)
                    
                    predictions = self.neural_network(images)
                    
                    # Calculate validation loss
                    val_batch_loss = self.loss_function(predictions, targets)
                    val_batch_losses.append(val_batch_loss)
                    
                    # Move predictions and labels to cpu for accuracy calculation
                    predictions = torch.max(predictions, dim=1)[1]
                    predictions = predictions.cpu().detach().numpy()
                    targets = targets.cpu().detach().numpy()
                    
                    # Calculate validation loss
                    val_batch_accuracy = sklearn.metrics.accuracy_score(y_pred=predictions, y_true=targets)
                    val_batch_accuracies.append(val_batch_accuracy)
            
            # Calculate epoch loss and accuracy        
            val_epoch_loss = float(torch.stack(val_batch_losses).mean())
            self.history["Validation Loss"].append(val_epoch_loss)
            
            val_epoch_accuracy = np.mean(val_batch_accuracies)
            self.history["Validation Accuracy"].append(val_epoch_accuracy)
            
    def test(self):
        """Test the model. Returns the test loss and test accuracy."""
        
        with torch.no_grad():
            print(" Testing...")
            
            test_batch_losses = []
            test_batch_accuracies = []
            
            self.neural_network.eval()
            
            for images, targets in tqdm(self.test_data):
                images = images.to(device)
                targets = targets.to(device)
                
                predictions = self.neural_network(images)
                
                # Calculate validation loss
                test_batch_loss = self.loss_function(predictions, targets)
                test_batch_losses.append(test_batch_loss)
                
                # Move predictions and labels to cpu for accuracy calculation
                predictions = torch.max(predictions, dim=1)[1]
                predictions = predictions.cpu().detach().numpy()
                targets = targets.cpu().detach().numpy()
                
                # Calculate validation loss
                test_batch_accuracy = sklearn.metrics.accuracy_score(y_pred=predictions, y_true=targets)
                test_batch_accuracies.append(test_batch_accuracy)
                
            # Calculate test loss and accuracy     
            test_loss = float(torch.stack(test_batch_losses).mean())
            test_accuracy = np.mean(test_batch_accuracies)
        
            return test_loss, test_accuracy
    
    def plot_history(self):
        """Plot the train and validation losses and accuracies for the latest training round."""
        
        if self.history == None:
            raise ValueError("Training history could not be found. Please train the model prior to plotting its losses.")
        
        _, axes = plt.subplots(2, 1, figsize=(15, 8))
        
        axes[0].plot(range(len(self.history["Train Loss"])), self.history["Train Loss"], label="Train")
        axes[0].plot(range(len(self.history["Validation Loss"])), self.history["Validation Loss"], label="Validation")
        axes[0].set_title(f"Train and Validation Losses for {self.history['Train Type']} Data")
        axes[0].set_xlabel("Epochs")
        axes[0].set_ylabel("Loss")
        axes[0].legend()
        
        axes[1].plot(range(len(self.history["Train Accuracy"])), self.history["Train Accuracy"], label="Train")
        axes[1].plot(range(len(self.history["Validation Accuracy"])), self.history["Validation Accuracy"], label="Validation")
        axes[1].set_title(f"Train and Validation Accuracies for {self.history['Train Type']} Data")
        axes[1].set_xlabel("Epochs")
        axes[1].set_ylabel("Accuracy")
        axes[1].legend()


### 1.6 Metrics

We define the metrics with which we measure the effectiveness of our attacks and defenses. These are the Attack Succes Rate (ASR), which measures how many of our adversarial samples resulted in wrongful classification, and the Clean Accuracy Drop, which measures the decrease in accuracy on a clean dataset when a clean model is fed adversarial samples.

In [6]:
def _count_non_source_misclassifications(targets: torch.Tensor, predictions: torch.Tensor, source_label, target_label):
    """Calculates and returns the number of classifications of images with a label that is not the source label nor the target label.
    
    Taken from the week 9 lab notebook (Backdoor Defenses, dr. Picek) and refactored."""
    
    sub_non_source_total = 0
    sub_misclassifications = 0
    
    sub_non_source_total_dict = {}
    sub_misclassification_dict = {}

    # Find all the images with a different label than the source or target label
    indices = torch.logical_and((targets != source_label), (targets != target_label)).nonzero(as_tuple=False).numpy()
    indices = indices.reshape(indices.shape[0])
    sub_non_source_total += indices.shape[0]

    # For all non-source and non-target label images, check if the prediction is equal to the target label
    for index in indices:
        target = targets[index].detach().cpu().numpy()
        prediction = predictions[index].detach().cpu().numpy()
        
        if str(target) in sub_non_source_total_dict:
            sub_non_source_total_dict[str(target)] += 1
        else:
            sub_non_source_total_dict[str(target)] = 1
        
        if prediction == target_label:
            sub_misclassifications += 1
            
            if str(target) in sub_misclassification_dict:
                sub_misclassification_dict[str(target)] += 1
            else:
                sub_misclassification_dict[str(target)] = 1
    
    return sub_misclassifications, sub_non_source_total, sub_misclassification_dict, sub_non_source_total_dict

def _count_source_specific_classifications(targets: torch.Tensor, predictions: torch.Tensor, source_label: int, target_label: int):
    """Calculates and returns the number of classifications of images with the source label.
    
    Taken from the week 9 lab notebook (Backdoor Defenses, dr. Picek) and refactored."""
    sub_total = 0
    sub_correct = 0
    
    # Find all the images with the source label
    indices = (targets == source_label).nonzero(as_tuple=False).numpy()
    indices = indices.reshape(indices.shape[0])
    sub_total += indices.shape[0]
    
    # For all source label images, check if the prediction is equal to the target label
    for i in indices:
        if predictions[i].detach().cpu().numpy() == target_label:
            sub_correct += 1
    
    return sub_correct, sub_total

def attack_success_rate(model: NeuralModel, adversarial_test_dataloader: DataLoader, target_label: int, source_label: int = None, verbose: bool = False) -> float:
    """Calculates and returns the Attack Success Rate.
    
    Taken from the week 9 lab notebook (Backdoor Defenses, dr. Picek) and refactored."""
    
    correct = 0
    total = 0
    non_source_total = 0
    misclassifications = 0
    
    non_source_total_dict = {}
    misclassification_dict = {}

    with torch.no_grad():
        model.neural_network.eval()

        for images, targets in tqdm(adversarial_test_dataloader):
            # Use poisoned test image to get predictions of backdoored model
            images = images.to(device)
            outputs = model.neural_network(images).detach()
            _, predictions = torch.max(outputs, dim=1)
            
            # If source agnostic attack
            if source_label is None:
                # For all test samples, check if the predicted label is equal to the target label
                for i in range(len(images)):
                    if targets[i] != target_label:
                        total += 1
                        
                        if predictions[i].detach().cpu().item() == target_label:
                            correct += 1
            # If source specific attack
            else:
                sub_correct, sub_total = _count_source_specific_classifications(targets, predictions, source_label, target_label)
                correct += sub_correct
                total += sub_total
                
                if verbose:
                    sub_misclassifications, sub_non_source_total, sub_misclassification_dict, sub_non_source_total_dict = _count_non_source_misclassifications(targets, predictions, source_label, target_label)
                    misclassifications += sub_misclassifications
                    non_source_total += sub_non_source_total
                    
                    for key in sub_misclassification_dict.keys():
                        if key in misclassification_dict:
                            misclassification_dict[key] += sub_misclassification_dict[key]
                        else:
                            misclassification_dict[key] = sub_misclassification_dict[key]
                            
                    for key in sub_non_source_total_dict.keys():
                        if key in non_source_total_dict:
                            non_source_total_dict[key] += sub_non_source_total_dict[key]
                        else:
                            non_source_total_dict[key] = sub_non_source_total_dict[key]
                            
        if verbose:
            for key in non_source_total_dict.keys():
                if key in misclassification_dict:
                    misclassification_dict[key] = round(misclassification_dict[key] / non_source_total_dict[key], 2)
                else:
                    misclassification_dict[key] = 0

    attack_success_rate = correct / total
    print(f"Attack Success Rate: {round(attack_success_rate, 2)}")
    
    if source_label and verbose:
        print(f"Number of Misclassifications:", misclassifications)
        print(f"Number of Images Not With Source Label:", non_source_total)
        print("Rate of Misclassification for Backdoored Images with Labels other than Source of Target:")
        
        for key, value in misclassification_dict.items():
            print(f" {key}: {value}")
        
        misclassification_rate = misclassifications / non_source_total
        print(f"False Positive Rate: {round(misclassification_rate, 2)}")
        
    return attack_success_rate

def clean_accuracy_drop(clean_model: NeuralModel, adversarial_model: NeuralModel) -> float:
    """Calculates and returns the Clean Accuracy Drop between a clean and adversarial model."""
    
    original_test_data_adversarial_model = copy(adversarial_model.test_data)
    adversarial_model.test_data = clean_model.test_data
    
    _, accuracy_clean_model = clean_model.test()
    _, accuracy_adversarial_model = adversarial_model.test()
    
    adversarial_model.test_data = original_test_data_adversarial_model
    
    clean_accuracy_drop = round(accuracy_clean_model - accuracy_adversarial_model, 2)
    print("Clean Accuracy Drop:", clean_accuracy_drop)
    
    return clean_accuracy_drop

### 1.7 Defenses
We define the defenses we want to use to protect our models from attacks.

In [7]:
class FinePruningDefense:
    def __init__(self, model: NeuralModel, block_to_prune: nn.Sequential) -> None:
        self.model = model
        self.block_to_prune = block_to_prune
        
        self.prune_rate = 0.2
        self.finetune_epochs = int(self.model.epochs * 0.1)
        
    def prune(self) -> None:     
        print(" Pruning...")
           
        for layer in self.block_to_prune.children():            
            if type(layer) is torch.nn.modules.conv.Conv2d:
                print(f"  Pruned layer {layer}")
                
                layer = torch.nn.utils.prune.l1_unstructured(module=layer, name="weight", amount=self.prune_rate)
        
    def finetune(self) -> None:      
        print(" Finetuning...")
         
        self.model.epochs = self.finetune_epochs
        self.model.train()

### 1.8 Aggregation Methods
We define a set of aggregation methods to aggregate the local models.
These include the [FedAvg](https://arxiv.org/pdf/1602.05629) and [Krum](https://proceedings.neurips.cc/paper_files/paper/2017/file/f4b9ec30ad9f68f89b29639786cb62ef-Paper.pdf) aggregation algorithms.

# SCHRIJF OVER JE IMPLEMENTATIES JIJ DWAAS

In [8]:
def fed_avg(all_models, base_model, verbose=True):
    """
    Perform FedAvg algorithm
    :param all_models list of state dicts, containing the locally trained parameters of the individual clients
    :param base_model state dict of arbitrary model, useful for knowing the names of all parameters and copying values of not 
    aggregated parameters
    :return state dict of aggregated model (obtained by FedAvg)
    
    Taken from the week 11 lab notebook (Federated Learning, dr. Picek) and refactored.
    """
    
    if verbose:
        src.Utils.print_timed(f'Aggregate {len(all_models)} models')
   
    result_state_dict = {name: torch.zeros_like(data) for name, data in base_model.items()} 
    n_models = len(all_models) 
    
    for state_dict in all_models:
        for layer_name in state_dict.keys():
            if layer_name in NAMES_OF_AGGREGATED_PARAMETERS:
                result_state_dict[layer_name] += state_dict[layer_name].to(device)
            else:
                result_state_dict[layer_name] += base_model[layer_name]
                
    for layer_name in result_state_dict.keys():
        result_state_dict[layer_name] = result_state_dict[layer_name] / n_models
    
    return result_state_dict

### 1.9 Federated Learning Setup
Finally, we initiate an FL setup using code from `src`.

In [18]:
cifar10_train = torchvision.datasets.CIFAR10(root="d:/Datasets", download=True, train=True, transform=FLSettings.transform_train)
cifar10_test = [p for p in torchvision.datasets.CIFAR10(root="d:/Datasets", download=True, train=False, transform=FLSettings.transform_test)]

train_data_by_labels, all_labels, all_training_images = src.Utils.sort_samples_by_labels(cifar10_train)

client_data_indices, main_labels_dict = src.Utils.create_client_distributions(
    total_client_number=FLSettings.total_client_count,
    iid_rate=FLSettings.iid_rate,
    samples_per_client=FLSettings.samples_per_client,
    all_labels=all_labels,
    train_data_by_labels=train_data_by_labels,
    all_training_images=all_training_images
)

all_training_data = [MyDataLoader(cifar10_train, indices, FLSettings.batch_size) for indices in tqdm(client_data_indices)]

global_model = NeuralModel("GlobalModel")

test_data = src.Utils.batchify(cifar10_test, FLSettings.test_batch_size, len(cifar10_test))
test_data = [(x.to(device), y.to(device)) for x, y in test_data]

blend_attack = BlendAttack(source_label=None, target_label=FLSettings.backdoor_target_class)
malignant_client_indexes = np.random.randint(low=0, high=FLSettings.total_client_count, size=FLSettings.malignant_client_count).tolist()
print("Indexes of Malignant Clients:", malignant_client_indexes)

for client_index in malignant_client_indexes:
    all_training_data[client_index] = src.PoisoningUtils.BackdoorData(
        data_loader=all_training_data[client_index],
        attack=blend_attack,
        pdr=FLSettings.malignant_client_poisoning_rate,
        BACKDOOR_TARGET_CLASS=FLSettings.backdoor_target_class,
        MEAN=FLSettings.data_mean,
        STD_DEV=FLSettings.data_std,
        COMPUTATION_DEVICE=device
    )

all_test_samples = []
for image, label in cifar10_test:
    if label == FLSettings.backdoor_target_class:
        continue
    
    all_test_samples.append((image, label))
    
backdoor_test_data_loader = DataLoader(all_test_samples, batch_size=FLSettings.test_batch_size, shuffle=True)
backdoor_test_data = src.PoisoningUtils.BackdoorData(
        data_loader=all_training_data[client_index],
        attack=blend_attack,
        pdr=1.0,
        BACKDOOR_TARGET_CLASS=FLSettings.backdoor_target_class,
        MEAN=FLSettings.data_mean,
        STD_DEV=FLSettings.data_std,
        COMPUTATION_DEVICE=device
    )
backdoor_test_data.cuda()

Files already downloaded and verified
Files already downloaded and verified


50000it [00:24, 2031.50it/s]


2024-05-29 12:25:32.906078: Samples from main class per client: 38
2024-05-29 12:25:32.906078: Samples from all classes per client: 346
2024-05-29 12:25:32.919044: Main label for clients: {0: 6, 1: 7, 2: 8, 3: 3, 4: 6, 5: 4} 


100%|██████████| 6/6 [00:01<00:00,  5.19it/s]


Files already downloaded and verified
Files already downloaded and verified
Indexes of Malignant Clients: [2, 4]


## Part 2: Q&A

### 2.1 Backdoor Attack in Federated Learning

#### 2.1.a 
Q: Implement a source-agnostic backdoor attack in FL using the
Blend attack with the Hello Kitty image. Below you find a list of parameters/settings to use for this attack. Limit your implementation to these values.
You will be investigating the performance of the attack with different number of
malicious clients in the network, i.e., 1, 2, or 3. In all cases, a total of 6 clients
compose the network. For example, in the first setting you have 1 malicious
and 5 benign clients. However, we ask you to select only a subset from all the
clients each round to perform the local training. This subset selection should be
random, but you are free how you implement this selection procedure. In the
end of the training, i.e., after 5 rounds of FL, plot the final ASR (y-axis) and
the global (poisoned) model’s task accuracy (also y-axis) versus the number of
malicious clients (x-axis). Either make two plots, one for ASR and one for accuracy, or combine the results in one. The following parameters/settings should
be used:
- **Model**: Load the pre-trained ResNet18Light model from the Federated
Learning tutorial.
- **Poisoning Rate**: Every malicious client should use a poisoning rate of
50% of the local dataset.
- **Global Aggregation Rounds**: 5.
- **Local Training Epochs**: 2.
- **Backdoor Target Class**: 0 (airplane).
- **Number Selected Clients Per Round**: 5.
- **Aggregation Method**: FedAvg.

In [21]:
local_models = []
for client_index in range(FLSettings.total_client_count):
    local_models.append(NeuralModel(f"LocalModel{client_index}"))

for aggregation_round in range(FLSettings.global_aggregation_rounds):
    all_trained_weights = []
    
    excluded_client_index = np.random.randint(low=0, high=FLSettings.total_client_count, size=1)
    print("Excluded client this round:", excluded_client_index)
    
    for client_index in range(FLSettings.total_client_count):        
        if client_index == excluded_client_index:
            continue
                
        src.Utils.print_timed(f'Client {client_index}')
        
        trained_weights = src.TrainingUtils.client_training(
            global_model_state_dict=global_model.neural_network_state_dict,
            local_model=local_models[client_index].neural_network,
            local_training_data=all_training_data[client_index],
            local_epochs=FLSettings.local_epochs,
            printing_prefix='\t',
            COMPUTATION_DEVICE=device
        )
        
        all_trained_weights.append(trained_weights)
    
    hash_values = src.ModelUtils.get_models_hash(all_trained_weights)
    aggregated_weights = fed_avg(all_trained_weights, global_model.neural_network_state_dict)
    src.ModelUtils.check_hashs(all_trained_weights, hash_values)
    
    global_model.neural_network.load_state_dict(aggregated_weights)
    global_model.neural_network_state_dict = aggregated_weights
    _ = src.ModelUtils.test(test_data, global_model.neural_network)

_ = src.ModelUtils.evaluate_model(
    model_to_evaluate=global_model.neural_network,
    test_data=test_data,
    backdoor_test_data=backdoor_test_data
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Excluded client this round: [4]
2024-05-29 12:46:41.744320: Client 0
2024-05-29 12:46:42.500312: 	local_epoch   0 | lr 0.2 | ms/batch 749.99| loss  4.82
2024-05-29 12:46:43.039776: 	local_epoch   1 | lr 0.2 | ms/batch 538.45| loss  1.26
2024-05-29 12:46:43.056770: Client 1
2024-05-29 12:46:43.605766: 	local_epoch   0 | lr 0.2 | ms/batch 543.96| loss  3.58
2024-05-29 12:46:44.147794: 	local_epoch   1 | lr 0.2 | ms/batch 542.03| loss  1.04
2024-05-29 12:46:44.161904: Client 2
2024-05-29 12:46:44.710938: 	local_epoch   0 | lr 0.2 | ms/batch 

A: 

#### 2.1.b 
Q: In question 2.1.a we asked you to use a subset of clients each round.
What do you think is the influence of the number of selected clients on the
performance of the backdoor attack in the federated setting? Share your conclusions.

A: 

#### 2.1.c
Q: In question 2.1.a we asked you to perform Federated Learning using
the FedAvg method for aggregation. Now using the same settings, but with just
1 malicious client, perform federated learning by making use of the **Krum** aggregation method. Plot the final ASR (y-axis) and global (poisoned) model task
accuracy (y-axis) for both the FedAvg and Krum methods (x-axis). Compare
the methods and share your conclusions on how both methods affect the ASR
and accuracy. For this homework question we ask you to assume 1 malicious
client, so f = 1. Also, only consider the subset of local models each round for
aggregation

A: 

### 2.2 Distributed Backdoor Attack

#### 2.2.a
Q: Implement the Distributed Backdoor Attack (DBA). The network
of clients is composed of three malicious clients and three benign clients. The
three malicious clients will split a red 6 × 6 square trigger vertically into three
equal parts. Use the following parameters/settings for your attack:
- **Model**: Load the pre-trained ResNet18Light model from the Federated Learning tutorial.
- **Poisoning Rate**: Every malicious client should use a poisoning rate of 50% of the local dataset.
- **Global Aggregation Rounds**: 5.
- **Local Training Epochs**: 2.
- **Backdoor Target Class**: 0 (airplane).
- **Number Selected Clients Per Round**: 6 (all).
- **Aggregation Method**: FedAvg.

Plot the final ASR (y-axis) and global (poisoned) model task accuracy (y-axis).
Compare your results with your plot from question 1(a). How does DBA perform
compared to the previous FL attack setting? Which FL attack setup performs
best and why do you think this is the case? Share your conclusions.

*Note*: In DBA, during the training phase, the trigger pattern is split between
the clients, but in testing time, you should test the attack (i.e., ASR and Model
Accuracy) with the complete original trigger and not tear it apart! Once FL
training is finished, we assume just one global model in test time and no clients
and servers.

A: 

### 2.3 Clipping Defense

#### 2.3.a
Q: Take exactly the same settings as question 2.1.a but limit it to just 1
malicious client. Use the blend attack and at each round let the malicious client
apply the scale update with a scaling factor of $\gamma = \frac{n}{\mu}$. Here $n$ is the number of
clients and $\mu$ is the learning rate which is also the number of malicious clients
and thus now set to 1. Finally, plot the ASR (y-axis) and global model accuracy
(y-axis) for just the “scaling approach” (x-axis). Store these results, as you will
re-use them in a new plot for the next question.

A: 

#### 2.3.b
Q: Enforce an upper boundary for the L2-norms of model updates, restricting the Euclidean distances between the global and the respective local
models. The model shall be down-scaled if the L2-norm exceeds the boundary
(clipping).

A challenge here is determining a suitable clipping boundary: if it is fixed and too
low, the aggregated model will be very similar to the global model. Otherwise,
if too high, this does not prevent scaling-based attacks.
Since the norm values of the models change during the training process and
become smaller when the model converges, we need to select the boundary dynamically. For that, compute the Euclidean distance from every model to the
aggregated one (from the previous round). Then, get the median, use that as
the clipping boundary, and clip all the models to have the same norm.5
Use the same setup as in question 2.3.a. This means you also apply the scaling
update at each round. Now at each aggregation step, you apply clipping defense
using a clipping boundary as explained above. For the final round, report a
figure similar to Figure 1, before and after clipping. Also, plot the ASR (y-axis)
and global model accuracy (y-axis) and add them to the plot of question 2.3.a
with ”clipping defense” below it (x-axis). Compare the results for only using
scaling and the case where you apply the defense. Was the defense effective
against the attack? Share your conclusions.

A: The best candidate layer to prune is most often the final convolution layer, so conv3. This is because deeper convolution layers most often learn the most features. Early convolution layers mostly learn basic features that are benign, but the deeper a layer is, and especially the more channels it has, the more likely it is that some channels/features are specialized in capturing information about the backdoor. Pruning the final layer would then mean removing channels/features dedicated to the backdoor, reducing the backdoor's effect.